In [30]:
import os, json
import pandas as pd

In [39]:
game_info = pd.read_parquet(GAME_INFO_PATH)
game_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8357 entries, 0 to 8356
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               8357 non-null   object        
 1   itad_uuid           8357 non-null   object        
 2   type                8002 non-null   category      
 3   achievements        8357 non-null   bool          
 4   mature              8357 non-null   bool          
 5   release_date        8048 non-null   datetime64[ns]
 6   rank                8357 non-null   int32         
 7   collected           8357 non-null   int32         
 8   steam_score         8357 non-null   int32         
 9   steam_review_count  8357 non-null   int32         
 10  has_release_date    8357 non-null   bool          
 11  review_tier         8357 non-null   category      
dtypes: bool(3), category(2), datetime64[ns](1), int32(4), object(2)
memory usage: 367.7+ KB


In [40]:
CACHE_PATH = "../data_raw/info/"
GAME_INFO_PATH = "../data/game_info.parquet"
game_early_access = []

for uid in game_info["itad_uuid"]:
    json_link = CACHE_PATH + uid + ".json"
    with open(json_link) as f:
        entry = json.load(f)
        game_early_access.append({
            "itad_uuid": uid,
            "early_access": entry.get("earlyAccess", {}),
            "peak_player_count": entry.get("players", {}).get("peak")
        })

game_early_access_df = pd.DataFrame(game_early_access)

In [41]:
game_early_access_df

,itad_uuid,early_access,peak_player_count
0,018d937f-157f-71fc-bc96-6376789a09aa,True,25612
1,01958f0e-3b2f-712f-9061-440be3587166,False,120488
2,018d937f-1eeb-71f9-97c7-a28eadd520c8,False,5
3,018d937f-0411-723e-9c22-89b8fc7a1504,False,2
4,018d937f-364f-7289-8036-daf6440c4411,False,197
...,...,...,...
8352,018d937f-1637-7076-a9cd-4222573b6681,False,810
8353,018d937e-fdbc-724f-b4ba-de64c4584dcb,False,61
8354,018d937f-0c2e-7260-a751-24587ce818ae,False,15
8355,018d937f-0a52-7160-987a-b931775fb32e,False,7


In [46]:
game_info.merge(game_early_access_df, on='itad_uuid', how="left",validate="one_to_one")

MergeError: Merge keys are not unique in either left or right dataset; not a one-to-one merge

In [44]:
# quick uniqueness checks
game_info["itad_uuid"] = game_info["itad_uuid"].astype(str).str.strip()
game_early_access_df["itad_uuid"] = game_early_access_df["itad_uuid"].astype(str).str.strip()

len_left = len(game_info)
len_right = len(game_early_access_df)
nuniq_left = game_info["itad_uuid"].nunique()
nuniq_right = game_early_access_df["itad_uuid"].nunique()
print(len_left, nuniq_left, len_right, nuniq_right)
# If len != nunique on either side, that side has duplicate keys.


8357 8353 8357 8353


In [45]:
dups_left = (game_info["itad_uuid"]
             .value_counts()
             .loc[lambda s: s>1]
             .index.tolist())

dups_right = (game_early_access_df["itad_uuid"]
              .value_counts()
              .loc[lambda s: s>1]
              .index.tolist())

print("Left dup count:", len(dups_left))
print("Right dup count:", len(dups_right))


Left dup count: 4
Right dup count: 4


In [59]:
game_early_access_df[game_early_access_df.duplicated()]

,itad_uuid,early_access,peak_player_count
2073,018d937f-6e8b-73d0-bfb7-ba50cd5fc7bd,False,51
2579,018d937e-f8cf-70ed-9f25-732f80db0f78,False,150
4739,018d937e-f046-71c3-8bca-f9ee16f9fb99,False,61
5130,018d937e-f4ea-713c-bd5a-28e490af0025,False,10


In [60]:
game_early_access_df.query("itad_uuid == '018d937f-6e8b-73d0-bfb7-ba50cd5fc7bd'")

,itad_uuid,early_access,peak_player_count
134,018d937f-6e8b-73d0-bfb7-ba50cd5fc7bd,False,51
2073,018d937f-6e8b-73d0-bfb7-ba50cd5fc7bd,False,51


In [54]:
game_info.query("itad_uuid == '018d937e-f4ea-713c-bd5a-28e490af0025'")

,title,itad_uuid,type,achievements,mature,release_date,rank,collected,steam_score,steam_review_count,has_release_date,review_tier
1621,CATS!,018d937e-f4ea-713c-bd5a-28e490af0025,game,True,False,2016-06-24,12112,3361,34,108,True,Poor
5130,Cats,018d937e-f4ea-713c-bd5a-28e490af0025,game,True,False,2016-06-24,12112,3361,34,108,True,Poor


In [61]:
dedup_right = (game_early_access_df.drop_duplicates(subset=["itad_uuid"], keep="last"))

In [62]:
dedup_right.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8353 entries, 0 to 8356
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   itad_uuid          8353 non-null   object
 1   early_access       8353 non-null   bool  
 2   peak_player_count  8353 non-null   int64 
dtypes: bool(1), int64(1), object(1)
memory usage: 203.9+ KB


In [58]:
dedup_right[dedup_right.duplicated()]

,itad_uuid,early_access,peak_player_count


In [63]:
game_info = game_info.drop_duplicates(subset=['itad_uuid'], keep='last')

In [64]:
merged = game_info.merge(
    dedup_right,
    on="itad_uuid",
    how="left",
    validate="one_to_one"
)

In [66]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8353 entries, 0 to 8352
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               8353 non-null   object        
 1   itad_uuid           8353 non-null   object        
 2   type                7998 non-null   category      
 3   achievements        8353 non-null   bool          
 4   mature              8353 non-null   bool          
 5   release_date        8044 non-null   datetime64[ns]
 6   rank                8353 non-null   int32         
 7   collected           8353 non-null   int32         
 8   steam_score         8353 non-null   int32         
 9   steam_review_count  8353 non-null   int32         
 10  has_release_date    8353 non-null   bool          
 11  review_tier         8353 non-null   category      
 12  early_access        8353 non-null   bool          
 13  peak_player_count   8353 non-null   int64       

In [67]:
merged.to_parquet("../data/game_info.parquet")